# Facial Classification Project
### Allison Christensen Thomas Heysel

##### To update the dataset in use, scroll down to the top of code cell 2 and update the input to the load_dataset function. The filename passed in must include 2 files named 'real' and 'fake' to properly function.

In [1]:
import cv2
import numpy as np
import time
from matplotlib import pyplot as plt
from os import listdir
from os.path import isfile, join

# Define required objects
sift = cv2.SIFT_create()
matcher = cv2.BFMatcher()

front_face_cascade = cv2.CascadeClassifier()
front_face_cascade.load(cv2.samples.findFile('haar_classifiers/haarcascade_frontalface_alt.xml'))
side_face_cascade = cv2.CascadeClassifier()
side_face_cascade.load(cv2.samples.findFile('haar_classifiers/haarcascade_profileface.xml'))

def load_dataset(filename='dataset'):
    # Assumes the data is stored in 2 files named dataset/real and dataset/fake
    # Returns 2 lists holding the real and fake images seperately 
    
    try:
        real_filenames = [filename+'/real/'+f for f in listdir(filename+'/real') if isfile(join(filename+'/real', f))]
        fake_filenames = [filename+'/fake/'+f for f in listdir(filename+'/fake') if isfile(join(filename+'/fake', f))]
    except FileNotFoundError as e:
        print(f"Error! Cannot read in dataset in the root file {filename}. Please ensure the file exists and "
              f"contains 2 folders named 'real' and 'fake'.")
        raise e
    
    real_images = [cv2.cvtColor(cv2.imread(f), cv2.COLOR_BGR2RGB) for f in real_filenames]
    fake_images = [cv2.cvtColor(cv2.imread(f), cv2.COLOR_BGR2RGB) for f in fake_filenames]

    return real_images, fake_images

def equalize_image(img):
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)

    # equalize the histogram of the Y channel
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])

    # convert the YUV image back to RGB format
    img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

    return img_output

def detect_face(img, side='front'):
    # Uses Haar Classifier to detect faces in an images and draw a bounding box around the facial region of interest
    # img : an image with a face
    # side : a string enum either "front" or "profile" descibing the type of image
    # Return : an image containing only the detected facial ROI for detected faces or [] if no face is detected
    
    # Convert the image to gray and equalize it. Helps to give better facial detection
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    equilized_gray_img = cv2.equalizeHist(gray_img) 
    
    # Detect the face and create and image of the detected face
    if side == 'front':
        face = front_face_cascade.detectMultiScale(equilized_gray_img, minSize=(150,150))
    else:
        face = side_face_cascade.detectMultiScale(equilized_gray_img, minSize=(150,150))
    for (x,y,w,h) in face:
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_color = img[y:y+h, x:x+w]
        roi_gray = equilized_gray_img[y:y+h, x:x+w]
    
    # Return the detected face. If no face was detected return an empty array
    try:
        roi_equalized = equalize_image(roi_color)
        return roi_equalized
    except UnboundLocalError as e:
        return []

def feature_matching(img1, img2):
    # Performs SIFT feature matching between 2 images and then uses Lowe's Ratio to filter out the bad matches
    # img1 : an RGB image
    # img2 : an RGB image
    # returns : the average of the two ratios  (number of matches / number of keypoints in img1) and
    #           (number of matches / number of keypoints in img2)  
    
    # Regular SIFT
    img1_keypoints, img1_descriptors = sift.detectAndCompute(img1, None)
    img2_keypoints, img2_descriptors = sift.detectAndCompute(img2, None)
    
    img1_with_keypoints = cv2.drawKeypoints(img1, img1_keypoints, np.array([]))
    img2_with_keypoints = cv2.drawKeypoints(img2, img2_keypoints, np.array([]))
    
    matches = matcher.knnMatch(img1_descriptors, img2_descriptors, k=2)
    matches_img = cv2.drawMatchesKnn(img1, img1_keypoints, img2, img2_keypoints, matches, np.array([]))
    
    # Lowe's Ratio    
    lowes_matches = [[match[0]] for match in matches if (match[0].distance / match[1].distance) < 0.8] 
    
    # Find the ratio
    r1 = len(lowes_matches) / len(img1_keypoints)
    r2 = len(lowes_matches) / len(img2_keypoints)
    avg = (r1 + r2) / 2
    
    return avg


def classification(imgC, imgL, imgR):
    # Classifies images as either real or fake
    # Takes 3 images as inputs
    # Returns True for real faces and False for fake face
    
    # The confidence measure of our assumption
    real_confidence = 0
    fake_confidence = 0
    
    # Real faces will require us to use a profile haar cascade for left and right images. But fake faces will 
    # require a frontal face cascade. 
    # Therefore for left and right images if we try a profile cascade and it detects a face it increases our 
    # confidence that its a real image. If we use a profile cascade and doesn't detect a face, we'll then try a 
    # frontal cascade. If it detects a face with the frontal cascade it increases our confidence that the image is 
    # fake.
    
    left_face = detect_face(imgL, 'profile')
    if len(left_face) == 0: # If no face is detected, try frontal cascade
        left_face = detect_face(imgL, 'front')
        if len(left_face) != 0: # Now a face is detected
            fake_confidence = fake_confidence + 1
    else:
        real_confidence = real_confidence + 1
    
    right_face = detect_face(imgR, 'profile')
    if len(right_face) == 0: # If no face is detected, try frontal cascade
        right_face = detect_face(imgL, 'front')
        if len(right_face) != 0: # Now a face is detected
            fake_confidence = fake_confidence + 1
    else:
        real_confidence = real_confidence + 1
    
    center_face = detect_face(imgC, 'front') # center always requires frontal cascade
    
    # Find matches between left and center and right and center faces
    num_pairs = 0
    ratio1 = 0 # Ratios represent the number of matches / the number of keypoints
    ratio2 = 0
    if len(left_face) != 0 and len(center_face) != 0:
        num_pairs = num_pairs + 1
        ratio1 = feature_matching(left_face, center_face)
    
    if len(right_face) != 0 and len(center_face) != 0: 
        num_pairs = num_pairs + 1
        ratio2 = feature_matching(right_face, center_face)
    
    # Calculate the average ratio of keypoint to matches between left and front comparison and right and front 
    # comparison. If the Haar Cascade was unable to detect faces in 2 or more of the images we cannot make a 
    # confident classification and deem it is 'Unclassifiable'
    try:
        avg_ratio = (ratio1 + ratio2) / num_pairs
    except ZeroDivisionError:
        return "Unclassifiable"
    
    if avg_ratio > 0.2:
        fake_confidence = fake_confidence + 1  
    else:
        real_confidence = real_confidence + 1
        
    if real_confidence > fake_confidence:
        return "Real"
    else:
        return "Fake"

In [2]:
##################################### Update dataset used here ###################################################
real_images, fake_images = load_dataset('dataset')
##################################################################################################################

unclassifiable = 0 # Counter of triplets that could not be classified
start = time.time() # Start timer

# Classify real images 
true_positive = 0
false_negative = 0
for i in range(0, len(real_images)-2, 3): 
    guess = classification(real_images[i], real_images[i + 1], real_images[i + 2])
    if guess == "Real":
        true_positive = true_positive + 1
    elif guess == "Fake":
        false_negative = false_negative + 1
    else:
        unclassifiable = unclassifiable + 1
        
# Classify fake images 
true_negative = 0
false_positive = 0
for i in range(0, len(fake_images)-2, 3): 
    guess = classification(fake_images[i], fake_images[i + 1], fake_images[i + 2])
    if guess == "Real":
        false_positive = false_positive + 1
    elif guess == "Fake":
        true_negative = true_negative + 1
    else:
        unclassifiable = unclassifiable + 1

end = time.time() # End timer
runtime = end - start

print('Confusion Matrix')
print(true_negative, ' | ', false_positive)
print('--------')
print(false_negative, ' | ', true_positive)

print(f'Number of triplets that were unclassifiable (this occurs if 2 or more of the images cannot detect'
      f' a face): {unclassifiable}')
print('Precision = ', true_positive/(true_positive + false_positive))
print('Recall = ', true_positive/(true_positive + false_negative))
print('Runtime: ', runtime, " ms")

Confusion Matrix
8  |  0
--------
0  |  11
Number of triplets that were unclassifiable (this occurs if 2 or more of the images cannot detect a face): 5
Precision =  1.0
Recall =  1.0
Runtime:  3.3856539726257324  ms
